In [4]:
import pandas as pd
data = pd.read_parquet('/Users/markshevchenkopu/BXL-Bouman-7/projects/08-DS-recruitment-challenge/transactions.parquet', engine='fastparquet')

In [5]:
from sklearn.utils import resample

df_1 = data[data['Returned'] == 1]
other_df = data[data['Returned'] == 0]  

# Undersample the majority class
df_1_training = resample(df_1, n_samples=45000, random_state=42)

df_other_undersampled_training = resample(other_df, n_samples=45000, random_state=42)

# Concatenate the undersampled majority class with the minority class
df_balanced = pd.concat([df_1_training, df_other_undersampled_training])

# Shuffle the concatenated DataFrame to randomize the order of samples
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

df_balanced['Shop_dupl']= df_balanced['Shop']
df_balanced['Shop_dupl2'] = df_balanced['Shop_dupl']

columns_to_keep = ['RevenueInclVAT', 'Returned', 'CostPriceExclVAT', 'OriginalSaleAmountInclVAT']

# Select columns to convert to categorical by removing columns to keep
columns_to_convert = df_balanced.columns.difference(columns_to_keep)

# Convert specified columns to categorical data type
df_balanced[columns_to_convert] = df_balanced[columns_to_convert].astype('category')


df_balanced.dtypes


Shop                         category
Order_Date_FK                category
ProductCode                  category
OriginalSaleAmountInclVAT     float64
CustomerID                   category
SaleDocumentNumber           category
Returned                        int64
RevenueInclVAT                float64
CostPriceExclVAT              float64
BrandName                    category
ModelGroup                   category
ProductGroup                 category
Shop_dupl                    category
Shop_dupl2                   category
dtype: object

In [7]:
import sqlite3 as sql

con = sql.connect('df_balanced.db')
df_balanced.to_sql('df_balanced', con, if_exists= 'replace', index=False)
connection = sql.connect('df_balanced.db')

pd.read_sql_query('''
WITH q_user AS (
    SELECT CustomerID, COUNT(SaleDocumentNumber) AS q_per_user
    FROM df_balanced
    GROUP BY CustomerID
),
q_invoice AS (
    SELECT CustomerID, SaleDocumentNumber, COUNT(SaleDocumentNumber) AS q_per_invoice
    FROM df_balanced
    GROUP BY SaleDocumentNumber
), combined AS (
                  SELECT q_user.CustomerID, q_user.q_per_user, q_invoice.SaleDocumentNumber, q_invoice.q_per_invoice
                  FROM q_user
                  JOIN q_invoice
                  ON q_invoice.CustomerID = q_user.CustomerID)
SELECT q_per_user, q_per_invoice, df_balanced.*
FROM combined
JOIN df_balanced
ON combined.CustomerID = df_balanced.CustomerID and combined.SaleDocumentNumber = df_balanced.SaleDocumentNumber
                  

                  ''', connection)

,q_per_user,q_per_invoice,Shop,Order_Date_FK,ProductCode,OriginalSaleAmountInclVAT,CustomerID,SaleDocumentNumber,Returned,RevenueInclVAT,CostPriceExclVAT,BrandName,ModelGroup,ProductGroup,Shop_dupl,Shop_dupl2
0,1,1,11,20210809,-4907581435789637196,9.99,-100037712728101204,24084458,0,8.49,3.838512,2929359717839159554,6742804956075612187,-5386210342051699146,11,11
1,2,2,31,20210621,-6661381600012052358,65.95,-1001204844398015219,23640815,0,65.95,22.283374,-4445228402666777776,3162564956579801398,-453682476182549203,31,31
2,2,2,31,20210621,-6661381600012052358,65.95,-1001204844398015219,23640815,0,65.95,22.283374,-4445228402666777776,3162564956579801398,-453682476182549203,31,31
3,1,1,4,20210628,-3593338250327416342,7.50,-1001473238972643331,23702632,0,7.50,1.936446,-8892132782483441489,6450924088821678307,5965103661511607192,4,4
4,2,2,29,20210704,-4725390224004084767,35.95,-100152308704859395,23758410,0,32.36,15.743982,-1671135878886367730,-4124706989267335995,-453682476182549203,29,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,4,3,82,20210720,6232877716898137454,95.95,998932113340652367,23939826,1,52.77,30.947133,6164254645951948400,-4124706989267335995,-453682476182549203,82,82
89996,4,3,82,20210720,6232877716898137454,95.95,998932113340652367,23939826,1,52.77,30.947133,6164254645951948400,-4124706989267335995,-453682476182549203,82,82
89997,1,1,14,20210607,-1481120730051027995,89.95,999149749246073924,23507069,1,89.95,34.927221,-7946506220046412185,3162564956579801398,-453682476182549203,14,14
89998,1,1,78,20210611,409770358902453964,26.99,999399182848491661,23547921,0,24.77,9.375500,-5813604796133860680,8676381074854750838,2500932515802331300,78,78


In [1]:

import matplotlib.pyplot as plt

# Import the scikitplot module
import scikitplot as skplt

